# READING THE DATASET

In [1]:

import os, types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xLMwOqjXYcPsiruuiyhBesen-NruLZFejZZ3W2pMOa7j',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'universityadmiteligibilitypredict-donotdelete-pr-kx4vyaapnlsosg'
object_key = 'Admission_Predict.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


# REMOVING UNWANTED COLUMNS

In [2]:
df.drop(["Serial No."],axis = 1,inplace = True)
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


# HANDLING MISSING VALUES IN THE DATASET

In [3]:
df.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

# FIND THE OUTLIERS AND REPLACE THEM

In [4]:
for i in df.columns:
    Q1 = df[i].quantile(0.25)
    Q3 = df[i].quantile(0.75)
    IQR = Q3-Q1
    upper_limit = Q3 + (1.5*IQR)
    lower_limit = Q1 - (1.5*IQR)
    df[i] = np.where(df[i]>=upper_limit,Q3 + (1.5*IQR),df[i])
    df[i] = np.where(df[i]<=lower_limit,Q1 - (1.5*IQR),df[i])

# SPLITTING THE DATA INTO DEPENDENT AND INDEPENDENT  VARIABLES

In [5]:
X = df.drop(columns=['Chance of Admit'])
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1.0
1,324.0,107.0,4.0,4.0,4.5,8.87,1.0
2,316.0,104.0,3.0,3.0,3.5,8.00,1.0
3,322.0,110.0,3.0,3.5,2.5,8.67,1.0
4,314.0,103.0,2.0,2.0,3.0,8.21,0.0
...,...,...,...,...,...,...,...
495,332.0,108.0,5.0,4.5,4.0,9.02,1.0
496,337.0,117.0,5.0,5.0,5.0,9.87,1.0
497,330.0,120.0,5.0,4.5,5.0,9.56,1.0
498,312.0,103.0,4.0,4.0,5.0,8.43,0.0


In [6]:
Y = df['Chance of Admit']
Y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

# SPLITTING INTO TRAINING AND TESTING DATASET

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2,random_state=90)

# BUILD THE MODEL

In [8]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# TRAIN THE MODEL

In [9]:
model.fit(x_train,y_train)

LinearRegression()

# TEST THE MODEL

In [10]:
y_predict = model.predict(x_test)
pd.DataFrame({'Actual':y_test.to_numpy(),'Predicted':y_predict})

,Actual,Predicted
0,0.56,0.616517
1,0.82,0.805325
2,0.71,0.682318
3,0.85,0.832543
4,0.47,0.487288
...,...,...
95,0.76,0.732623
96,0.69,0.671654
97,0.67,0.649773
98,0.76,0.793692


# EVALUATE THE PERFORMANCE OF MODEL USING METRICS

In [11]:
from sklearn.metrics import r2_score
r2_score(y_test,y_predict)

0.8724323452611149

# IBM DEPLOYMENT

In [12]:
!pip install -U ibm-watson-machine-learning

In [13]:
from ibm_watson_machine_learning import APIClient
import json

# AUTHENTICATE AND SET SPACE

In [14]:
wml_credentials = {
    "apikey":"0cJXVMJod7os4JhIiWZORtvpfuoRr8aPtQMSthvh76hb",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [15]:
wml_client = APIClient(wml_credentials)

In [16]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  --------------------------------------  ------------------------
ID                                    NAME                                    CREATED
21b21952-1c72-4eba-9e5c-8a0bdc0c32f8  University-Admit-Eligibility-Predictor  2022-11-15T16:05:58.092Z
------------------------------------  --------------------------------------  ------------------------


In [17]:
SPACE_ID = "21b21952-1c72-4eba-9e5c-8a0bdc0c32f8"

In [18]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [19]:
wml_client.software_specifications.list(500)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

# SAVE AND DEPLOY THE MODEL

In [20]:
import sklearn
sklearn.__version__

'1.0.2'

In [21]:
MODEL_NAME = "University-Admit-Eligibility-Predictor"
DEPLOYMENT_NAME = "University-Admit-Eligibility-Predictor"
DEMO_MODEL = model

In [22]:
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [23]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME : MODEL_NAME,
    wml_client.repository.ModelMetaNames.TYPE : 'scikit-learn_1.0',
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : software_spec_uid,
}

In [24]:
model_details = wml_client.repository.store_model(
    model = DEMO_MODEL,
    meta_props = model_props,
    training_data = x_train,
    training_target = y_train
)

In [25]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'Chance of Admit',
  'schemas': {'input': [{'fields': [{'name': 'GRE Score', 'type': 'float64'},
      {'name': 'TOEFL Score', 'type': 'float64'},
      {'name': 'University Rating', 'type': 'float64'},
      {'name': 'SOP', 'type': 'float64'},
      {'name': 'LOR ', 'type': 'float64'},
      {'name': 'CGPA', 'type': 'float64'},
      {'name': 'Research', 'type': 'float64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-15T17:52:04.660Z',
  'id': '1c5a916d-ec71-44c2-b56f-6e9507c446f7',
  'modified_at': '2022-11-15T17:52:07.408Z',
  'name': 'University-Admit-Eligibility-Predictor',
  'owner': 'IBMid-665002HVJ7',
  'resource_key': '8390778a-db7a-4dce-b559-7c3a4eba1eae',
  'space_id': '21b21952-1c72-4eba-9e5c-8a0bdc0c32f8'},
 'system': {'warni

In [26]:
model_id = wml_client.repository.get_model_id(model_details)
model_id

'1c5a916d-ec71-44c2-b56f-6e9507c446f7'

In [27]:
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME : DEPLOYMENT_NAME,
    wml_client.deployments.ConfigurationMetaNames.ONLINE : {}
}

In [28]:
deployment = wml_client.deployments.create(
    artifact_uid = model_id,
    meta_props = deployment_props
)



#######################################################################################

Synchronous deployment creation for uid: '1c5a916d-ec71-44c2-b56f-6e9507c446f7' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='1b254abd-eaec-4f59-9e86-a3674f3eea4c'
------------------------------------------------------------------------------------------------


